**DEM comparisons and sources**



**USGS 1-meter DEM:** https://catalog.data.gov/dataset/usgs-national-elevation-dataset-ned-1-meter-downloadable-data-collection-from-the-national-map-

*need to figure out how to download DEM data from the USGS url into notebook*

In [ ]:
#Import necessary modules
#from osgeo import gdal
#import numpy as np
#import matplotlib.pyplot as plt

#from pygeotools.lib import iolib, warplib, geolib, timelib, malib

1-arcsec (30-m) USGS National Elevation Dataset derived from digitized contour maps with 1970 source date (I merged several tiles and clipped to a large extent around Rainier)

**GEE DEM from USGS Elevation**
url : https://developers.google.com/earth-engine/datasets/catalog/USGS_NED

details: USGS National Elevation Dataset 1/3 arc-second


line of code in GEE..

**Load USGS elevation data into GEE**

In [ ]:
#exploring a DEM USGS dataset below...

var dataset = ee.Image('USGS/NED');
print (dataset);
var elevation = dataset.select('elevation');
var elevationVis = {
  min: 0.0,
  max: 4000.0,
  gamma: 1.6,
};
#set your map center
Map.setCenter(-120.073, 37.822, 6);

#add map layer of elevation data if you wish:
Map.addLayer(elevation, elevationVis, 'Elevation');

**Create a hillshade**

In [ ]:
#Hillshade example.  This is a demonstration of computing
#a hillshade from terrain data and displaying multiple
#layers based on multiple view geometries.  Hillshade
#creation is also provided by ee.Terrain.hillshade().

#Define a function to convert from degrees to radians.

function radians(img) {
  return img.toFloat().multiply(Math.PI).divide(180);
}

#Define a function to compute a hillshade from terrain data
#for the given sun azimuth and elevation.

function hillshade(az, ze, slope, aspect) {
  
#Convert angles to radians.
  
  var azimuth = radians(ee.Image(az));
  var zenith = radians(ee.Image(ze));

#Note that methods on images are needed to do the computation.
#i.e. JavaScript operators (e.g. +, -, /, *) do not work on images.
#The following implements:
#Hillshade = cos(Azimuth - Aspect) * sin(Slope) * sin(Zenith) +
#    cos(Zenith) * cos(Slope)
  
  return azimuth.subtract(aspect).cos()
    .multiply(slope.sin())
    .multiply(zenith.sin())
    .add(
      zenith.cos().multiply(slope.cos()));
}

#Compute terrain meaasures from the SRTM DEM.

var terrain = ee.Algorithms.Terrain(ee.Image('USGS/NED'));
var slope = radians(terrain.select('slope'));
var aspect = radians(terrain.select('aspect'));

#select the hillshade band from the terrain USGS image:
var dem = terrain.select('hillshade')
print (dem);

#if you want to add dem for whole country as as map layer:
Map.addLayer(dem, {}, 'dem');


**Clip hillshade to state of CA**

In [ ]:
#pull data from US Census
var featureCollection = ee.FeatureCollection("TIGER/2016/States");
#isolate California shapefile by name
var region = featureCollection.filterMetadata('NAME', 'equals', 'California');

#if you want to add the shapefile of CA as a layer to map:
Map.addLayer(region, {}, 'cali')

#clipping hillshade to california shape and adding as a map layer:
Map.addLayer(dem.clip(region),{}, 'finaldem');


**Add Central Valley watershed boundary**

In [ ]:
#add shapefile asset to map as a layer
#need to import it into code line as a feature and then add to map as layer.

var cv_outline = ee.FeatureCollection('users/rachaelhaggen/cv_outline');
Map.addLayer(cv_outline, {color: '937134'}, 'cv_outline');